<a href="https://colab.research.google.com/github/Fatehab/machine-learning/blob/main/PublicdatasetBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['LC_CTYPE'] = 'en_US.UTF-8'

In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import string
import numpy as np
import spacy 
import matplotlib.pyplot as plt
import seaborn as sns
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

In [ ]:
#Data Input
path ="/content/drive/MyDrive/smalldata/cybertroll.csv"
df = pd.read_csv(path)
df

,Unnamed: 0,content,annotation,label
0,1,Get fucking real dude.,"{'notes': '', 'label': ['1']}",1
1,2,She is as dirty as they come and that crook ...,"{'notes': '', 'label': ['1']}",1
2,3,why did you fuck it up. I could do it all day...,"{'notes': '', 'label': ['1']}",1
3,4,Dude they dont finish enclosing the fucking s...,"{'notes': '', 'label': ['1']}",1
4,5,WTF are you talking about Men? No men thats n...,"{'notes': '', 'label': ['1']}",1
...,...,...,...,...
7995,7996,It sucks but at least my car wasn't stolen. ri...,"{'notes': '', 'label': ['0']}",0
7996,7997,It may even suck ass.,"{'notes': '', 'label': ['0']}",0
7997,7998,just because of a dang song. Over it now but d...,"{'notes': '', 'label': ['0']}",0
7998,7999,I dunno I hate it though. Someone at work ga...,"{'notes': '', 'label': ['0']}",0


In [ ]:
#removing annotation and unamed column from dataframe
df.drop('annotation', axis=1, inplace=True)
df

,Unnamed: 0,content,label
0,1,Get fucking real dude.,1
1,2,She is as dirty as they come and that crook ...,1
2,3,why did you fuck it up. I could do it all day...,1
3,4,Dude they dont finish enclosing the fucking s...,1
4,5,WTF are you talking about Men? No men thats n...,1
...,...,...,...
7995,7996,It sucks but at least my car wasn't stolen. ri...,0
7996,7997,It may even suck ass.,0
7997,7998,just because of a dang song. Over it now but d...,0
7998,7999,I dunno I hate it though. Someone at work ga...,0


In [ ]:
#Pre-processing 
#Converting dataset to lowercase letters
df['content'] = df['content'].apply(lambda x: " ".join(x.lower () for x in x.split()))
#Removing Puncuation from dataset
df['content'] = df['content'].str.replace('[^\w\s]','')

def removeSpecialCharacter(v):
    c = "".join([r for r in v if ('A' <= r <= 'Z') or ('a' <= r <= 'z') or (r == " ")])
    return c

df['content']= df['content'].apply(lambda x: removeSpecialCharacter (str(x)))
df['content'].replace('', np.nan, inplace=True)

df.dropna(subset=['content'], inplace=True)

#removing numbers from text
def remove_digits(txt):
    txt_nodigit = "".join([char for char in txt if not char.isdigit()])
    return txt_nodigit


df['content'] = df['content'].apply(lambda x: remove_digits(x))
df

<ipython-input-7-b8f3e9d99886>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^\w\s]','')


,Unnamed: 0,content,label
0,1,get fucking real dude,1
1,2,she is as dirty as they come and that crook re...,1
2,3,why did you fuck it up i could do it all day t...,1
3,4,dude they dont finish enclosing the fucking sh...,1
4,5,wtf are you talking about men no men thats not...,1
...,...,...,...
7995,7996,it sucks but at least my car wasnt stolen righ...,0
7996,7997,it may even suck ass,0
7997,7998,just because of a dang song over it now but da...,0
7998,7999,i dunno i hate it though someone at work gave ...,0


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report


# Load pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize text and generate BERT embeddings
X = df['content'].values
X_tokens = tokenizer.batch_encode_plus(X, padding=True, truncation=True, max_length=256, return_tensors='pt')
input_ids = X_tokens['input_ids']
attention_mask = X_tokens['attention_mask']
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)[0]
features = last_hidden_states[:,0,:].numpy()

# Split the data into train and test sets
labels = df['label'].values
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Train the SVM classifier on the BERT features
svm_classifier = SVC(kernel='linear', C=1, gamma='auto')
svm_classifier.fit(X_train, Y_train)

# Evaluate the SVM classifier on the test set
svm_accuracy = svm_classifier.score(X_test, Y_test)
print(f'SVM Accuracy: {svm_accuracy:.4f}')

test_preds = svm_classifier.predict(X_test)
test_accuracy = svm_classifier.score(X_test, Y_test)
test_report = classification_report(Y_test, test_preds)

# Generate predictions on the training set
train_preds = svm_classifier.predict(X_train)

# Calculate accuracy score and classification report
train_accuracy = svm_classifier.score(X_train, Y_train)
train_report = classification_report(Y_train, train_preds)

# Print the results
print(f'Training Accuracy: {train_accuracy:.4f}')
print('Classification Report on Training Set:')
print(train_report)

print(f'Testing Accuracy: {test_accuracy:.4f}')
print('Classification Report on Testing Set:')
print(test_report)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SVM Accuracy: 0.6032
Training Accuracy: 0.7299
Classification Report on Training Set:
              precision    recall  f1-score   support

           0       0.73      0.73      0.73      2819
           1       0.73      0.73      0.73      2778

    accuracy                           0.73      5597
   macro avg       0.73      0.73      0.73      5597
weighted avg       0.73      0.73      0.73      5597

Testing Accuracy: 0.6032
Classification Report on Testing Set:
              precision    recall  f1-score   support

           0       0.59      0.61      0.60      1181
           1       0.61      0.60      0.60      1218

    accuracy                           0.60      2399
   macro avg       0.60      0.60      0.60      2399
weighted avg       0.60      0.60      0.60      2399



In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

# Load pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize text and generate BERT embeddings
X = df['content'].values
X_tokens = tokenizer.batch_encode_plus(X, padding=True, truncation=True, max_length=256, return_tensors='pt')
input_ids = X_tokens['input_ids']
attention_mask = X_tokens['attention_mask']
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)[0]
features = last_hidden_states[:,0,:].numpy()

# Split the data into train and test sets
labels = df['label'].values
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Train the MLP classifier on the BERT features
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
mlp_classifier.fit(X_train, Y_train)

test_preds = mlp_classifier.predict(X_test)
test_accuracy = mlp_classifier.score(X_test, Y_test)
test_report = classification_report(Y_test, test_preds)

# Evaluate the MLP classifier on the test set
mlp_accuracy = mlp_classifier.score(X_test, Y_test)
print(f'MLP Accuracy: {mlp_accuracy:.4f}')

print(f'Testing Accuracy: {test_accuracy:.4f}')
print('Classification Report on Testing Set:')
print(test_report)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


MLP Accuracy: 0.6824
Testing Accuracy: 0.6824
Classification Report on Testing Set:
              precision    recall  f1-score   support

           0       0.72      0.58      0.64      1181
           1       0.66      0.78      0.71      1218

    accuracy                           0.68      2399
   macro avg       0.69      0.68      0.68      2399
weighted avg       0.69      0.68      0.68      2399



In [ ]:
# Generate predictions on the training set
train_preds = mlp_classifier.predict(X_train)

# Calculate accuracy score and classification report
train_accuracy = mlp_classifier.score(X_train, Y_train)
train_report = classification_report(Y_train, train_preds)

# Print the results
print(f'Training Accuracy: {train_accuracy:.4f}')
print('Classification Report on Training Set:')
print(train_report)

Training Accuracy: 0.9971
Classification Report on Training Set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2819
           1       1.00      1.00      1.00      2778

    accuracy                           1.00      5597
   macro avg       1.00      1.00      1.00      5597
weighted avg       1.00      1.00      1.00      5597



In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Binarizer

# Load pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize text and generate BERT embeddings
X = df['content'].values
X_tokens = tokenizer.batch_encode_plus(X, padding=True, truncation=True, max_length=256, return_tensors='pt')
input_ids = X_tokens['input_ids']
attention_mask = X_tokens['attention_mask']
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)[0]
features = last_hidden_states[:,0,:].numpy()

# Convert BERT embeddings to non-negative integers
binarizer = Binarizer(threshold=0)
binarized_features = binarizer.fit_transform(features)

# Split the data into train and test sets
labels = df['label'].values
X_train, X_test, Y_train, Y_test = train_test_split(binarized_features, labels, test_size=0.3, random_state=42)

# Train the Naive Bayes classifier on the binarized features
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, Y_train)

# Evaluate the Naive Bayes classifier on the test set
nb_accuracy = nb_classifier.score(X_test, Y_test)
print(f'Naive Bayes Accuracy: {nb_accuracy:.4f}')

# Generate predictions on the test set
test_preds = nb_classifier.predict(X_test)

# Calculate accuracy score and classification report
test_accuracy = nb_classifier.score(X_test, Y_test)
test_report = classification_report(Y_test, test_preds)

# Print the results
print(f'Testing Accuracy: {test_accuracy:.4f}')
print('Classification Report on Testing Set:')
print(test_report)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Naive Bayes Accuracy: 0.6128
Testing Accuracy: 0.6128
Classification Report on Testing Set:
              precision    recall  f1-score   support

           0       0.59      0.68      0.63      1181
           1       0.64      0.55      0.59      1218

    accuracy                           0.61      2399
   macro avg       0.62      0.61      0.61      2399
weighted avg       0.62      0.61      0.61      2399



In [ ]:
# Print out the classification report on the training set
Y_train_pred = nb_classifier.predict(X_train)
print(classification_report(Y_train, Y_train_pred))

              precision    recall  f1-score   support

           0       0.61      0.66      0.63      2819
           1       0.62      0.57      0.59      2778

    accuracy                           0.61      5597
   macro avg       0.61      0.61      0.61      5597
weighted avg       0.61      0.61      0.61      5597



In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import classification_report

# Load pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize text and generate BERT embeddings
X = df['content'].values
X_tokens = tokenizer.batch_encode_plus(X, padding=True, truncation=True, max_length=256, return_tensors='pt')
input_ids = X_tokens['input_ids']
attention_mask = X_tokens['attention_mask']
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)[0]
features = last_hidden_states[:,0,:].numpy()

# Split the data into train and test sets
labels = df['label'].values
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Train the XGBoost classifier on the BERT features
dtrain = xgb.DMatrix(X_train, label=Y_train)
params = {'objective': 'multi:softmax', 'num_class': len(np.unique(Y_train))}
xgb_classifier = xgb.train(params, dtrain)

# Evaluate the XGBoost classifier on the test set
dtest = xgb.DMatrix(X_test)
Y_test_pred = xgb_classifier.predict(dtest)
xgb_accuracy = np.mean(Y_test_pred == Y_test)
print(f'XGBoost Accuracy: {xgb_accuracy:.4f}')
# Print out the classification report on the testing set
print(classification_report(Y_test, Y_test_pred))

# Print out the classification report on the training set
dtrain_pred = xgb_classifier.predict(dtrain)
print(classification_report(Y_train, dtrain_pred))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


XGBoost Accuracy: 0.6636
              precision    recall  f1-score   support

           0       0.65      0.67      0.66      1181
           1       0.67      0.65      0.66      1218

    accuracy                           0.66      2399
   macro avg       0.66      0.66      0.66      2399
weighted avg       0.66      0.66      0.66      2399

              precision    recall  f1-score   support

           0       0.91      0.92      0.91      2819
           1       0.91      0.90      0.91      2778

    accuracy                           0.91      5597
   macro avg       0.91      0.91      0.91      5597
weighted avg       0.91      0.91      0.91      5597



In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report

# Load pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize text and generate BERT embeddings
X = df['content'].values
X_tokens = tokenizer.batch_encode_plus(X, padding=True, truncation=True, max_length=256, return_tensors='pt')
input_ids = X_tokens['input_ids']
attention_mask = X_tokens['attention_mask']
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)[0]
features = last_hidden_states[:,0,:].numpy()

# Split the data into train and test sets
labels = df['label'].values
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Train the AdaBoost classifier on the BERT features
ada_classifier = AdaBoostClassifier(n_estimators=100)
ada_classifier.fit(X_train, Y_train)

# Evaluate the AdaBoost classifier on the test set
ada_accuracy = ada_classifier.score(X_test, Y_test)
print(f'AdaBoost Accuracy: {ada_accuracy:.4f}')

# Print out the classification report on the training set
Y_train_pred = ada_classifier.predict(X_train)
print(classification_report(Y_train, Y_train_pred))

# Evaluate the AdaBoost classifier on the test set
Y_test_pred = ada_classifier.predict(X_test)
print(classification_report(Y_test, Y_test_pred))



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


AdaBoost Accuracy: 0.6186
              precision    recall  f1-score   support

           0       0.73      0.72      0.73      2819
           1       0.72      0.73      0.73      2778

    accuracy                           0.73      5597
   macro avg       0.73      0.73      0.73      5597
weighted avg       0.73      0.73      0.73      5597

              precision    recall  f1-score   support

           0       0.61      0.63      0.62      1181
           1       0.63      0.61      0.62      1218

    accuracy                           0.62      2399
   macro avg       0.62      0.62      0.62      2399
weighted avg       0.62      0.62      0.62      2399

